<a href="https://colab.research.google.com/github/Murcicrum/Scrapers-de-inmobiliarias/blob/main/scraper_mercadolibre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

import time
from numpy import random

import pandas as pd

import re, json
import csv

Función auxiliar que reintenta el request por si hay problemas de conexión a internet

In [ ]:
def parse_url( url:str ):
    '''
    Dado un url devuelve objeto parseado de BeautifulSoup
    '''
    while True:
        try:
            response = requests.get( url )
            break
        except:
            pass
            #print('Conexión a publicación malió sal, reintentando....', url)
    
    if response.status_code != 200:
      print('Request malió sal.', '\tStatus code', response.status_code, url)
      return None
    
    return bs( response.content )

#Levanto publicaciones

##### Defino los urls de las busquedas

In [ ]:
URL_SEARCH_DEPTOS = 'https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_'
URL_SEARCH_CASAS  = 'https://inmuebles.mercadolibre.com.ar/casas/alquiler/capital-federal/_Desde_'
URL_SEARCH_PHS    = 'https://inmuebles.mercadolibre.com.ar/ph/alquiler/capital-federal/_Desde_'

##### Scraper que extrae los links a las publicaciones

In [ ]:
def get_links(url_search:str, N:int=5) -> list:
  '''
  Devuelve lista con los links a publicaciones de alquileres, dado:
    url_search  string de búsqueda con el formato que especifica numero de página
    N           numero de páginas a recorrer.Si se llega al límite se detiene.
  '''
  urls = []

  for n in range(N):
    
    url_request = url_search + str( n*48+1 )
    
    soup = parse_url( url_request )
    if not soup:    continue
    else:           print('Extrayendo links de página ',n)
    
    tags = soup.find_all( name='a', 
                          attrs={'class':'ui-search-result__content ui-search-link'} )
    
    urls += [ t['href'] for t in tags ]
    
    next_page_button = soup.find( name='li', attrs={'class':'andes-pagination__button--next'} )

    if not next_page_button:
        print('Se alcanzó la última página.')
        break

  
  print(f'Se extrajeron {len(urls)} links')
  return urls

Extraigo los links a las publicaciones \\


In [ ]:
url_deptos = get_links(URL_SEARCH_DEPTOS,50)

Extrayendo links de página  0
Extrayendo links de página  1
Extrayendo links de página  2
Extrayendo links de página  3
Extrayendo links de página  4
Extrayendo links de página  5
Extrayendo links de página  6
Extrayendo links de página  7
Extrayendo links de página  8
Extrayendo links de página  9
Extrayendo links de página  10
Extrayendo links de página  11
Extrayendo links de página  12
Extrayendo links de página  13
Extrayendo links de página  14
Extrayendo links de página  15
Extrayendo links de página  16
Extrayendo links de página  17
Extrayendo links de página  18
Extrayendo links de página  19
Extrayendo links de página  20
Extrayendo links de página  21
Extrayendo links de página  22
Extrayendo links de página  23
Extrayendo links de página  24
Extrayendo links de página  25
Extrayendo links de página  26
Extrayendo links de página  27
Extrayendo links de página  28
Extrayendo links de página  29
Extrayendo links de página  30
Extrayendo links de página  31
Extrayendo links d

In [ ]:
url_casas = get_links(URL_SEARCH_CASAS,10)

Extrayendo links de página  0
Extrayendo links de página  1
Extrayendo links de página  2
Extrayendo links de página  3
Se alcanzó la última página.
Se extrajeron 184 links


In [ ]:
url_phs = get_links(URL_SEARCH_PHS,10)

Extrayendo links de página  0
Extrayendo links de página  1
Extrayendo links de página  2
Extrayendo links de página  3
Extrayendo links de página  4
Se alcanzó la última página.
Se extrajeron 222 links



#Levanto datos

##### Funciones que se encargan de scrapear cada publicación

In [ ]:
def find_pub_date(url: str):
    '''
    Dado el URL de una publicación activa de MeLi, devuelve la fecha de publicación utilizando la API.
    '''  
    URL_API = 'https://api.mercadolibre.com/items/'
    PATTERN = "MLA-\d+"
    pub_id = re.findall( re.compile( PATTERN ), url )[0]
    pub_id = pub_id.replace('-','')
      
    while True:
        try:
            response = requests.get( URL_API + pub_id )
            break
        except Exception as e:
            pass
            #print('Conexión a API malió sal, reintentando...')
    pub_json = response.json()
      
    return pub_json['start_time']


def get_data(urls:list, filename:str='', n:int=30) -> list:
    '''
    Dada una lista de links con publicaciones de inmuebles de mercadolibre
    Devuelve lista de diccionarion con datos del precio, superficie, ubicación...
    '''
    data_list = []
 
    for i,url in enumerate(urls):
        soup = parse_url( url )
        if not soup:    continue

        pub_finalizada = soup.find(name='div', attrs={'class':'ui-pdp-container__row ui-pdp-container__row--item-status-message'})
        if pub_finalizada:  
            print('Link',i, ' Publicación Finalizada', url)
            continue
        
##### EMPIEZA LA EXTRACCIÓN DE DATOS #####
        d_data = {}

#### Estos los saco de scrapear el html de la publicación                
### ALGUNOS DATOS
        to_extract = {'sp_tot':     'Superficie total', 
                      'sp_cub':     'Superficie cubierta', 
                      'nu_ambs':    'Ambientes', 
                      'nu_dorms':   'Dormitorios', 
                      'pr_exp':     'Expensas'}
        tags_table = soup.find_all(name='tr',attrs={'class':'andes-table__row'})
        
        for key, title in to_extract.items():
            d_data[key] = None
            for tag in tags_table:
                text = tag.text
                if text.startswith(title):
                   value = text.replace(title,'').split()[0]
                   value = value.split('.')[0]
                   d_data[key] = int(value)

### PRECIO
        tag_price = soup.find(name='span', attrs={'class':'price-tag-fraction'})
        d_data['pr_valor'] = int( tag_price.text.replace('.','') )

        tag_currency = soup.find(name='span',attrs={'class':'price-tag-symbol'})
        d_data['pr_moneda'] = tag_currency.text 

### UBICACION
        tag_address = soup.find_all(name='p', 
                                    attrs={'class':'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-media__title'})[-1]
        d_data['ub_calle'] = tag_address.text.replace(', Capital Federal, Capital Federal', '')
                               
        tags_script = soup.find_all(name='script')
        pattern = re.compile( '"location":\{(.*?)\}' )
        locations = re.findall( pattern, tags_script[-1].text )
        locations = json.loads( '{'+locations[0]+'}' )           
        try:
            d_data['ub_lat'] = locations['latitude']
            d_data['ub_lon'] = locations['longitude']
        except:
            d_data['ub_lat'] = None
            d_data['ub_lon'] = None
        

### FECHA PPUBLICACION
#### Esto lo obtengo usando la API
        d_data['fe_pub'] = find_pub_date( url )      

### URL
        d_data['url'] = url


        data_list.append(d_data)        
        
##### GUARDADO DE DATOS #####
        if filename and ( i%n==0 or i==len(urls)-1 ):
            with open(filename, 'a') as f:
                writer = csv.DictWriter(f, fieldnames=d_data.keys())
                if i==0:
                    print('Los datos se guardarán en', filename)
                    writer.writeheader()
                    writer.writerow( data_list[0] )
                else:
                    print(i, 'Guardando...')
                    if i%n==0:
                        for data in data_list[-n:]:
                            writer.writerow(data)
                    elif i==len(urls)-1:
                        for data in data_list[-(i%n):]:
                            writer.writerow(data)
        
    return data_list

##### Monto el drive para guardar los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

PATH_DRIVE = 'drive/MyDrive/Colab Notebooks/Espacios públicos TDP/'

n_guardar = 10

Mounted at /content/drive/


##### Extraigo los datos de las publicaciones


Para deptos

In [ ]:
FILENAME_DEPTOS = PATH_DRIVE + 'meli_deptos.csv'

data_deptos = get_data(url_deptos, FILENAME_DEPTOS, n_guardar)

Los datos se guardarán en drive/MyDrive/Colab Notebooks/Espacios públicos TDP/meli_deptos.csv
10 Guardando...
20 Guardando...
30 Guardando...
40 Guardando...
50 Guardando...
60 Guardando...
70 Guardando...
80 Guardando...
90 Guardando...
100 Guardando...
110 Guardando...
120 Guardando...
130 Guardando...
140 Guardando...
150 Guardando...
160 Guardando...
170 Guardando...
No se encontró latitud-longitud en https://departamento.mercadolibre.com.ar/MLA-936014574-alq-semipiso-4-amb-ccochera-y-dep-a-estrenar-_JM#position=35&search_layout=grid&type=item&tracking_id=ff56edf4-f6dd-442a-b15d-2066def78dfd
180 Guardando...
190 Guardando...
200 Guardando...
210 Guardando...
220 Guardando...
230 Guardando...
240 Guardando...
250 Guardando...
260 Guardando...
270 Guardando...
280 Guardando...
290 Guardando...
Link 297  Publicación Finalizada https://departamento.mercadolibre.com.ar/MLA-1107975602-espectacular-departamento-2-amb-al-frente-a-estrenar-_JM#position=10&search_layout=grid&type=item&tracki

Para casas



In [ ]:
FILENAME_CASAS  = PATH_DRIVE + 'meli_casas.csv'
data_casas = get_data(url_casas, FILENAME_CASAS, n_guardar)

Los datos se guardarán en drive/MyDrive/Colab Notebooks/Espacios públicos TDP/meli_casas.csv
10 Guardando...
20 Guardando...
30 Guardando...
40 Guardando...
50 Guardando...
60 Guardando...
70 Guardando...
80 Guardando...
90 Guardando...
100 Guardando...
110 Guardando...
120 Guardando...
130 Guardando...
140 Guardando...
150 Guardando...
160 Guardando...
170 Guardando...
180 Guardando...
183 Guardando...


Para phs

In [ ]:
FILENAME_PHS = PATH_DRIVE + 'meli_phs.csv'
data_phs = get_data(url_casas, FILENAME_PHS, n_guardar)

Los datos se guardarán en drive/MyDrive/Colab Notebooks/Espacios públicos TDP/meli_phs.csv
10 Guardando...
20 Guardando...
30 Guardando...
40 Guardando...
50 Guardando...
60 Guardando...
70 Guardando...
80 Guardando...
90 Guardando...
100 Guardando...
110 Guardando...
120 Guardando...
130 Guardando...
140 Guardando...
150 Guardando...
160 Guardando...
170 Guardando...
180 Guardando...
183 Guardando...


#Merge de todos los datos

In [ ]:
df_deptos = pd.read_csv( filename_deptos)
df_casas  = pd.read_csv( filename_casas )
df_phs    = pd.read_csv( filename_phs   )

In [ ]:
df_deptos['inm_tipo']= 'depto'
df_casas['inm_tipo'] = 'casa'
df_phs['inm_tipo']   = 'ph'

Junto los tres

In [ ]:
df_merge = pd.concat([df_deptos,df_casas,df_phs])
df_merge.shape

(2374, 13)

Lo guardo en un csv

In [ ]:
df_merge.to_csv( PATH_DRIVE + 'meli.csv' )

In [ ]:
#chequeo si los puedo leer
dfdf = pd.read_csv(PATH_DRIVE+'meli.csv')
dfdf.shape